December 04, 2024

Goal: Explore the benchmark circuits in `feynman/benchmarks/qasm` and determine their important properties

In [1]:
cd ..

/Users/abhi/Documents/ubc-coursework/cpsc-513/cpsc-513-project


First, let's try to get all the files in the dir automatically using python

In [2]:
benchmark_dir_path = "../feynman/benchmarks/qasm/"

import glob
file_path_list = glob.glob(benchmark_dir_path + "*.qasm")
print(file_path_list) 

['../feynman/benchmarks/qasm/adder_8.qasm', '../feynman/benchmarks/qasm/mod_adder_1048576.qasm', '../feynman/benchmarks/qasm/barenco_tof_5.qasm', '../feynman/benchmarks/qasm/cycle_17_3.qasm', '../feynman/benchmarks/qasm/gf2^4_mult.qasm', '../feynman/benchmarks/qasm/gf2^128_mult.qasm', '../feynman/benchmarks/qasm/mod_mult_55.qasm', '../feynman/benchmarks/qasm/barenco_tof_4.qasm', '../feynman/benchmarks/qasm/gf2^32_mult.qasm', '../feynman/benchmarks/qasm/gf2^64_mult.qasm', '../feynman/benchmarks/qasm/csum_mux_9.qasm', '../feynman/benchmarks/qasm/gf2^9_mult.qasm', '../feynman/benchmarks/qasm/hwb8.qasm', '../feynman/benchmarks/qasm/rc_adder_6.qasm', '../feynman/benchmarks/qasm/qft_4.qasm', '../feynman/benchmarks/qasm/barenco_tof_3.qasm', '../feynman/benchmarks/qasm/grover_5.qasm', '../feynman/benchmarks/qasm/barenco_tof_10.qasm', '../feynman/benchmarks/qasm/mod_adder_1024.qasm', '../feynman/benchmarks/qasm/qcla_com_7.qasm', '../feynman/benchmarks/qasm/mod5_4.qasm', '../feynman/benchmarks/q

In [3]:
path_0 = file_path_list[0]
path_0

'../feynman/benchmarks/qasm/adder_8.qasm'

In [4]:
path_0.lstrip(benchmark_dir_path)

'dder_8.qasm'

In [6]:
path_0.replace(benchmark_dir_path, '', 1).replace('.qasm', '', 1)

'adder_8'

Ok. We can get the benchmark name using the method above. How can we collect the relevant metrics?

In [8]:
from qiskit import QuantumCircuit
circ_0 = QuantumCircuit.from_qasm_file(path_0)

In [9]:
print(circ_0.num_qubits, circ_0.num_ancillas)

15 0


In [11]:
circ_0.count_ops()

OrderedDict([('h', 40), ('cx', 20), ('ccx', 10)])

In [17]:
sum(circ_0.count_ops().values())
circ_0.count_ops().keys()

odict_keys(['h', 'cx', 'ccx'])

In [19]:
"_".join(list(circ_0.count_ops().keys()))

'h_cx_ccx'

In [13]:
circ_0.depth()

24

Ok, we now have everything we need to get the important metrics from the circuits. let's write the loop and build the dictionary.

In [3]:
file_path_list

['../feynman/benchmarks/qasm/adder_8.qasm',
 '../feynman/benchmarks/qasm/mod_adder_1048576.qasm',
 '../feynman/benchmarks/qasm/barenco_tof_5.qasm',
 '../feynman/benchmarks/qasm/cycle_17_3.qasm',
 '../feynman/benchmarks/qasm/gf2^4_mult.qasm',
 '../feynman/benchmarks/qasm/gf2^128_mult.qasm',
 '../feynman/benchmarks/qasm/mod_mult_55.qasm',
 '../feynman/benchmarks/qasm/barenco_tof_4.qasm',
 '../feynman/benchmarks/qasm/gf2^32_mult.qasm',
 '../feynman/benchmarks/qasm/gf2^64_mult.qasm',
 '../feynman/benchmarks/qasm/csum_mux_9.qasm',
 '../feynman/benchmarks/qasm/gf2^9_mult.qasm',
 '../feynman/benchmarks/qasm/hwb8.qasm',
 '../feynman/benchmarks/qasm/rc_adder_6.qasm',
 '../feynman/benchmarks/qasm/qft_4.qasm',
 '../feynman/benchmarks/qasm/barenco_tof_3.qasm',
 '../feynman/benchmarks/qasm/grover_5.qasm',
 '../feynman/benchmarks/qasm/barenco_tof_10.qasm',
 '../feynman/benchmarks/qasm/mod_adder_1024.qasm',
 '../feynman/benchmarks/qasm/qcla_com_7.qasm',
 '../feynman/benchmarks/qasm/mod5_4.qasm',
 '..

In [ ]:
from keys import IBMQ_API
from qiskit import QuantumCircuit, transpile
from qiskit.qasm2 import QASM2ParseError
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(channel="ibm_quantum", token=IBMQ_API, instance="ibm-q/open/main")
ibm_backend = service.backend("ibm_sherbrooke")

In [11]:
circ_info_dict = {
    'name': [],
    'n_gates_original': [],
    'n_gates_transpiled': [],
    'depth_original': [],
    'depth_transpiled': [],
    'gate_set': [],
    'n_qubits_original': [],
    'n_qubits_transpiled': [],
    'n_ancilla_original': [],
    'n_ancilla_transpiled': []
}

for file_path in file_path_list[:3]:
    # build the circuit from the .qasm file
    try:
        circ = QuantumCircuit.from_qasm_file(file_path)

        # extract the name of the benchmark
        circ_info_dict['name'].append(
            file_path.replace(benchmark_dir_path, '', 1).replace('.qasm', '', 1)
        )

        # get the metrics from the original "uncompiled" circuit
        circ_op_dict = circ.count_ops()
        circ_info_dict['gate_set'].append('_'.join(list(circ_op_dict.keys())))
        
        circ_info_dict['n_qubits_original'].append(circ.num_qubits)
        circ_info_dict['n_gates_original'].append(sum(circ_op_dict.values()))
        circ_info_dict['depth_original'].append(circ.depth())
        circ_info_dict['n_ancilla_original'].append(circ.num_ancillas)
        
        # transpile the circuit to the IBM backend (ibm_sherbrooke) and get the metrics from the
        # transpiled circuit
        circ_hw = transpile(circ, ibm_backend)
        
        circ_hw_op_dict = circ_hw.count_ops()
        circ_info_dict['n_qubits_transpiled'].append(circ_hw.num_qubits)
        circ_info_dict['n_gates_transpiled'].append(sum(circ_hw_op_dict.values()))
        circ_info_dict['depth_transpiled'].append(circ_hw.depth())
        circ_info_dict['n_ancilla_transpiled'].append(circ_hw.num_ancillas)
    
    except QASM2ParseError:
        print(f"Unable to parse {file_path} due to QASM2ParseError.")

Unable to parse ../feynman/benchmarks/qasm/mod_adder_1048576.qasm due to QASM2ParseError.


In [12]:
import pandas as pd
pd.DataFrame.from_dict(circ_info_dict)

,name,n_gates_original,n_gates_transpiled,depth_original,depth_transpiled,gate_set,n_qubits_original,n_qubits_transpiled,n_ancilla_original,n_ancilla_transpiled
0,adder_8,330,5724,78,1428,h_cx_ccx_x,24,127,0,0
1,barenco_tof_5,50,986,38,548,h_ccx,9,127,0,0
